In [1]:
# need to be in the feature-circuit branch of pyGSTi.
from __future__ import division, print_function, absolute_import, unicode_literals
from ipyparallel import Client
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import fractional_matrix_power
from scipy.linalg import expm
import pygsti
from pygsti.extras import newrb as prb
from pygsti.extras import rb

##### Commented out below to make it work on Tim's machine
rw = Client()
#rw = Client(profile="mpi2")
size = len(rw)
dw = rw[:]
dw.scatter('rank', range(size), flatten=True)
dw['size'] = size
cwd = os.getcwd()
dw['cwd'] = cwd

In [6]:
#%%px 
from __future__ import division, print_function, absolute_import, unicode_literals

# ----- File title (get inserted as the start of the filenames) ----- #
title = 'sectors-one-qubit-bad'

# Whether to add the data to the end of already-existing files
appenddata=False

# Whether to run crb as well as prb simulations
runcrb = False

# ----- The specification of the perfect device ---- #

# The gateset (probably best not to change without talking to Tim)
gllist = ['Gi','Gh','Gp','Gcphase'] #+ ['X','Y','Z']

# Fill in manual connectivity here (linear chain is available as a default, below)
availability = {} 

#If false, full connectivity (or whatever is given by "availability" above).
linearchain = True

# A list of the different numbers of qubits to run sim on. n = 1 cause a failure (will fix soon).
nlist = [8] #[2,3,4]#,5,6,7,8]

# ----- The specification of the perfect device ---- #

# Uncomment for an error model whereby every gate is followed by a coherent error with fixed rot. angle
#errortype = 'fixed-over-rotation'
# The over-rotation parameter [the over-rotations are of the form exp(-i theta*pi*(unit-vec*paulis)/2)],
# as a function of n
def thetawithn(n):
    return 0.01 

# Uncomment for an error model whereby every gate is followed by a coh. error with error increasing with q. number
#errortype = 'variable-over-rotation'
# The min and max theta, with the thetas taken to vary linearly between these values.
def maxminthetawithn(n):
    mintheta = 0.000 
    maxtheta = 0.015
    return mintheta, maxtheta

# Uncomment for an error model one qubit has fairly large coherent Z-rotation errors after every gate
errortype = 'one-qubit-bad'
badqubit = 0
badqubittheta = 0.03

# ----- The specification of the PRB parameters ---- #

# The num of seqs sampled at each length, on each core (having 200-300 total seqs/length is a good idea )
def numseqswithn(n): return 40
# Function gives the lengths used for each number of qubits (allows different lengths as a function of n).
def rblengths(n,rbtype='prb'):
    if rbtype == 'prb': 
        if n == 2:
            #return [0,1,2,3,4,5,6,7,8,9,10,20,40]
            return [0,10,20,30,40,50,60,70,80,90,100,200,400,600,800,1000]
        if n == 3:
            #return [0,1,2,3,4,5,6,7,8,9,10,20,40]
            return [0,10,20,30,40,50,60,70,80,90,100,200,300,400,600,800]
        if n == 4:
            #return [0,1,2,3,4,5,6,7,8,9,10,20,40]
            return [0,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600]
        if n == 5:
            return [0,10,20,30,40,50,60,70,80,90,100,150,200,250,300,400]
        if n == 6:
            return [0,10,20,30,40,50,60,70,80,90,100,125,150,200,250,300]
        if n == 7:
            return [0,10,20,30,40,50,60,70,80,90,100,125,150,175,200,250]
        if n == 8:
            return [0,10,20,30,40,50,60,70,80,90,100,110,125,150,175,200,400,600,800,1000]
        
    if rbtype == 'crb': 
        if n == 2:
            return [1,2,3,4,5,6,7,8,9,10,11,20,40,60,80,100]
        if n == 3:
            return [1,2,3,4,5,6,7,8,9,10,11,20,30,40,50,60]
        if n == 4:
            return [1,2,3,4,5,6,7,8,9,10,11,15,20,25,30,40]
        if n == 5:
            return [1,2,3,4,5,6,7,8,9,10,11,13,15,20,25,30]
        if n == 6:
            return [1,2,3,4,5,6,7,8,9,10,11,13,15,18,20,25]
        if n == 7:
            return [1,2,3,4,5,6,7,8,9,10,11,13,14,15,17,20]
        if n == 8:
            return [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    
# Pick the desired sampling-distribution labels. If "weights", then each qubit is looked at in turn (in a
# random order), the possible gates still available is considered and if a two-qubit gate is still possible
# then we decide with probability twoprobwithn(n) (see below) to implement one of these. If so, one of the
# available two-qubit gates is picked uniformly at random, if not one of the single qubit gates is picked uniformly
# at random. If "sectors", then sectors of two-qubit gates are defined (in a fixed way -- see code below) containing
# two-qubit gates that can be applied in parallel. Then one of the sectors is picked at uniformly at random, and each
# two-qubit gate in it is implemented with a probability twoprobwithn(n) (see below).

# sampler = 'weights'
sampler = "sectors"

# If sampler = "weights", this is the probability that a two-qubit gate is implemented on any given
# qubit, *if*, given what has been chosen in the layer already, this is still possible. (so, it roughly
# corresponds to the probability that any given two-qubit gate is implemented in the layer). If sample =
# "sectors", this is the probability that each two-qubit gate is implemented in the sampled sector of
# two-qubit gates. In that case, the chance of implementing any given two qubit gate is then this value
# multiplied by the number of sectors a two-qubit gate appears in (fixed to 1 here) divided by the
# number of sectors (fixed to 2 here).
def twoqprobwithn(n): return 1/2

#
alternatewithlocal=False
#
localtype='paulis'

In [7]:
#%%px
from __future__ import division, print_function, absolute_import, unicode_literals
import numpy as np
from scipy.linalg import fractional_matrix_power
from scipy.linalg import expm
import pygsti
from pygsti.extras import newrb as prb
from pygsti.extras import rb
import os
#np.random.seed(size*rank + 134567)

In [16]:
#%%px 

x = np.array([[0,1.],[1.,0]])
y = np.array([[0,-1j],[1j,0]])
z = np.array([[1.,0],[0,-1.]])

pspec = {}
lengths_prb = {}
lengths_crb = {}
K = {}
sampler_args = {}

for i in range(0,len(nlist)):
    
    availability = {}
    n = nlist[i]
    if linearchain == True:
        connectedlist = []
        for i in range(0,n-1):
            connectedlist.append((i,i+1))
        availability['Gcphase'] = connectedlist
        
    pspec[n] = pygsti.obj.ProcessorSpec(n, gllist, availability=availability, verbosity=0)    

    # Over-rotation-like-error with fixed rotation-angle-over/pi theta
    pspec[n].add_std_model('fixed-over-rotation', parameterization="static", sim_type="svmap")

    theta = thetawithn(n)
    for gate in list(pspec[n].models['fixed-over-rotation'].gates.keys()):
        if gate.name == 'Gp':
            uerror = expm(-1j*np.pi*theta*z/2)
        elif gate.name == 'Gcphase':
            uerror = np.kron(expm(-1j*np.pi*theta*z/2),expm(-1j*np.pi*theta*z/2))
        elif gate.name == 'Gh':
            uerror = expm(-1j*np.pi*theta*(x + z)/(2*np.sqrt(2)))
        else:
            uerror = np.identity(2)
        tunitary = pspec[n].models['target'].gates[gate].embedded_gate
        tunitary = tunitary.convert_to_matrix(tunitary)
        pspec[n].models['fixed-over-rotation'].gates[gate] = np.dot(uerror,tunitary)
        
            
    # Over-rotation-like-error with rotation-angle-over/pi theta varying linearly with qubit num.
    pspec[n].add_std_model('variable-over-rotation', parameterization="static", sim_type="svmap")
    mintheta,maxtheta = maxminthetawithn(n)
    theta = np.linspace(mintheta,maxtheta,n)
    for gate in list(pspec[n].models['variable-over-rotation'].gates.keys()):
        if gate.name == 'Gp':
            uerror = expm(-1j*np.pi*theta[gate.qubits[0]]*z/2)
        elif gate.name == 'Gcphase':
            uerror = np.kron(expm(-1j*np.pi*theta[gate.qubits[0]]*z/2),expm(-1j*np.pi*theta[gate.qubits[1]]*z/2))
        elif gate.name == 'Ch':
            uerror = expm(-1j*np.pi*theta[gate.qubits[0]]*(x + z)/(2*np.sqrt(2)))
        else:
            uerror = np.identity(2)
        tunitary = pspec[n].models['target'].gates[gate].embedded_gate
        tunitary = tunitary.convert_to_matrix(tunitary)
        pspec[n].models['variable-over-rotation'].gates[gate] = np.dot(uerror,tunitary)
        
    # Over-rotation-like-error on all single-qubit gates on a particular qubit.
    pspec[n].add_std_model('one-qubit-bad', parameterization="static", sim_type="svmap")
    
    for gate in list(pspec[n].models['one-qubit-bad'].gates.keys()):
        if gate.name == 'Gp' and (badqubit in gate.qubits):
            uerror = expm(-1j*np.pi*badqubittheta*z/2)
        elif gate.name == 'Gcphase':
            uerror = np.identity(4)
        elif gate.name == 'Ch' and (badqubit in gate.qubits):
            uerror = expm(-1j*np.pi*badqubittheta*(x + z)/(2*np.sqrt(2)))
        else:
            uerror = np.identity(2)
        tunitary = pspec[n].models['target'].gates[gate].embedded_gate
        tunitary = tunitary.convert_to_matrix(tunitary)
        pspec[n].models['one-qubit-bad'].gates[gate] = np.dot(uerror,tunitary)

    # The PRB and CRB sampling parameters
    lengths_prb[n] = rblengths(n,rbtype='prb')
    lengths_crb[n] = rblengths(n,rbtype='crb')
    K[n] = numseqswithn(n)
    
    if sampler == 'weights':
        sampler_args[n]={'two_qubit_weighting' : twoqprobwithn(n)}
    if sampler == 'sectors':
        sector1 = [pygsti.objects.Label('Gcphase',(2*i,2*i+1)) for i in range(0,int(np.floor((n-2)/2))+ 1)]
        sector2 = [pygsti.objects.Label('Gcphase',(2*i+1,2*i+2)) for i in range(0,int(np.floor((n-1)/2)))]
        sectors = [sector1,sector2]
        sampler_args[n]={'sectors' : sectors, 'two_qubit_prob' : twoqprobwithn(n)}

In [81]:
c = prb.sample_prb_circuit(pspec[8],0,sampler=sampler, sampler_args=sampler_args[8], alternatewithlocal=alternatewithlocal,
                           localtype=localtype) 

# Fails with and without this.
c.replace_gatename('I','Gi')
print(c._tup_dirty)

True


In [82]:
# No fail if this is called
#c._reinit_base()

In [83]:
# This is where the fail happens
c.simulate(pspec[8].models['one-qubit-bad'])

KeyError: Label('H', 0)

In [17]:
print("ProcessorSpecs created")

ProcessorSpecs created


In [18]:
#pspec[8].compilations

In [19]:
#%%px

def simulate_rb(pspec, k, lengths, filename, mname, rbtype='PRB', sampler=None, sampler_args=None, appenddata=False,
                alternatewithlocal=False, localtype='primitives', returndata=False):
    
    import time
    
    n = pspec.number_of_qubits
    
    if not appenddata:
        with open(filename,'w') as f:
            f.write('#length Pm circuitdepth 2Qgatecount\n')
    
    # Temporary aux-file writing
    auxfn = filename[:(len(filename)-4)] + '_aux.txt'
    with open(auxfn,'w') as f:
        f.write('# Aux\n')
            
    if returndata:
        pm_lst = []
        circuit_2Qcount_lst = []
        circuit_depth_lst = []

        for l in lengths:
            pm_lst.append([])
            circuit_2Qcount_lst.append([])
            circuit_depth_lst.append([])
            
    counter = 0
    for i in range(0,k):
        for j in range(0,len(lengths)):
            l = lengths[j]
            passed = False
            
            with open(auxfn,'a') as f:
                    f.write('Start generating RB circuit... ')
            
            if rbtype == 'PRB':
                t1 = time.time()
                                  
                while not passed:
                    
                    try:
                        c = prb.sample_prb_circuit(pspec,l,sampler=sampler, sampler_args=sampler_args,   
                                                   alternatewithlocal=alternatewithlocal,localtype=localtype) 
                        c.replace_gatename('I','Gi')
                        print(c)
                        passed = True
                        counter += 1
                    except:
                        counter += 1
                        if counter > 100:
                            break
                        
            if rbtype == 'CRB':
                c = prb.sample_crb_circuit(ds,l)
                passed = True
            
            t2 = time.time()
            with open(auxfn,'a') as f:
                f.write('Finshed. Circuit depth is {}. Time taken is {} \n'.format(c.depth(),t1-t2))                                    
            
            if passed:
                circuit_2Qcount = c.twoqubit_gatecount()
                circuit_depth = c.depth()
                                                   
                t3 = time.time()
                with open(auxfn,'a') as f:
                     f.write('Starting simulation... ')
                                                   
                out = c.simulate(pspec.models[mname])
                pm = out['0'*n]     
                
                t4 = time.time()
                                                   
                with open(auxfn,'a') as f:
                    f.write('Finished. Take takin is {}\n'.format(t4-t3))
                                                   
                if returndata:
                    circuit_2Qcount_lst[j].append(circuit_2Qcount)
                    circuit_depth_lst[j].append(circuit_depth)
                    pm_lst[j].append(pm)
                with open(filename,'a') as f:
                    f.write('{} {} {} {}\n'.format(l,pm,circuit_depth,circuit_2Qcount)) 
                                                   
            if not passed:
                with open(filename,'a') as f:
                    f.write('F F F F\n'.format(l,pm,circuit_depth,circuit_2Qcount))
    if returndata:
        return [pm_lst, circuit_2Qcount_lst, circuit_depth_lst, passed]

pm_prb = {}
pm_crb = {}
circuit_2Qcount_prb = {}
circuit_2Qcount_crb = {}
circuit_depth_prb = {}
circuit_depth_crb = {}
passed_prb = {}
passed_crb = {}

cwd = os.getcwd()

for i in range(0,len(nlist)):
    n = nlist[i]
    #filename =  cwd + '/output/sims/data/'+title+'_{}'.format(rank)+ '_{}qubits'.format(n)
    filename =  cwd + '/output/sims/data/'+title+'_{}'.format(0)+ '_{}qubits'.format(n)
    
    simulate_rb(pspec[n], K[n], lengths_prb[n], filename+'_prb.txt', errortype, 'PRB', sampler,
                sampler_args[n], appenddata=appenddata, alternatewithlocal=alternatewithlocal, 
                localtype=localtype, returndata=False)

    if runcrb:
        simulate_rb(pspec[n], K[n], lengths_crb[n], filename+'_crb.txt', errortype, 'CRB', returndata=True)

Qubit 0 ---|Gh |-|Gp |-|Gp |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gh |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gh |-|Gh |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gh |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-Gcphase:0:1|-||Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |

KeyError: Label('H', 0)

### Code that predicts the decay rate *if* we had full twirling at every stage

In [ ]:
predictedp = {}
samples = 50
for n in nlist:
    for i in range(0,samples):
         layer = prb.sample_primitives_circuit(ds,1,sampler=sampler, sampler_args=sampler_args).get_layer(0)

for ind in range(size):
    for n in nlist:
        print(rw[ind]['pm_prb'][n])

nlist =rw[0]['nlist']

results_PRB = {}
results_CRB = {}
asp_PRB = {}
asp_CRB = {}
varsp_PRB = {}
varsp_CRB = {}
lengths_PRB = {}
lengths_CRB = {}
circuit_2Qcount_PRB = {}
circuit_2Qcount_CRB = {}
circuit_depth_PRB = {}
circuit_depth_CRB = {}
a2QC_PRB = {}
a2QC_CRB = {}
adepth_PRB = {}
adepth_CRB = {}
var2QC_PRB = {}
var2QC_CRB = {}
vardepth_PRB = {}
vardepth_CRB = {}

for n in nlist:
    results_PRB[n] = np.concatenate([np.array(rw[ind]['pm_prb'][n]) for ind in range(size) if rw[ind]['passed_prb'][n]], axis=1)
    results_CRB[n] = np.concatenate([np.array(rw[ind]['pm_crb'][n]) for ind in range(size) if rw[ind]['passed_crb'][n]], axis=1)
    lengths_PRB[n] = np.array(rw[0]['lengths_prb'][n])
    lengths_CRB[n] = np.array(rw[0]['lengths_crb'][n])
    circuit_depth_PRB[n] = np.concatenate([np.array(rw[ind]['circuit_depth_prb'][n]) for ind in range(size) if rw[ind]['passed_prb'][n]], axis=1)
    circuit_depth_CRB[n] = np.concatenate([np.array(rw[ind]['circuit_depth_crb'][n]) for ind in range(size) if rw[ind]['passed_crb'][n]], axis=1)
    circuit_2Qcount_PRB[n] = np.concatenate([np.array(rw[ind]['circuit_2Qcount_prb'][n]) for ind in range(size) if rw[ind]['passed_prb'][n]], axis=1)
    circuit_2Qcount_CRB[n] = np.concatenate([np.array(rw[ind]['circuit_2Qcount_crb'][n]) for ind in range(size) if rw[ind]['passed_crb'][n]], axis=1)
    asp_PRB[n] =  np.mean(results_PRB[n],axis=1)
    asp_CRB[n] =  np.mean(results_CRB[n],axis=1)
    a2QC_PRB[n] =  np.mean(circuit_2Qcount_PRB[n],axis=1)
    a2QC_CRB[n] =  np.mean(circuit_2Qcount_CRB[n],axis=1)
    adepth_PRB[n] =  np.mean(circuit_depth_PRB[n],axis=1)
    adepth_CRB[n] =  np.mean(circuit_depth_CRB[n],axis=1)
    varsp_PRB[n] =  np.std(results_PRB[n],axis=1)**2
    varsp_CRB[n] =  np.std(results_CRB[n],axis=1)**2
    var2QC_PRB[n] =  np.std(circuit_2Qcount_PRB[n],axis=1)**2
    var2QC_CRB[n] =  np.std(circuit_2Qcount_CRB[n],axis=1)**2
    vardepth_PRB[n] =  np.std(circuit_depth_PRB[n],axis=1)**2
    vardepth_CRB[n] =  np.std(circuit_depth_CRB[n],axis=1)**2

** Below is code for creating figures. It is commented out to avoid getting to lots of analysis (instead use the analysis notebook). **

import matplotlib.pyplot as plt
%matplotlib inline

for n in nlist:
    plt.figure(figsize=(15,7))
    plt.title("{} qubit RB".format(n))
    plt.plot(lengths_CRB[n]-1, asp_CRB[n],'.-',label='CRB')
    plt.plot(lengths_PRB[n], asp_PRB[n],'.-',label='PRB')
    plt.legend()
    plt.ylim(0,1)
    
    filename =  cwd + "/fig_outs/"+title+" - {} qubit RB.pdf".format(n)
    plt.savefig(filename)
    plt.show()

for n in nlist:
    plt.figure(figsize=(15,7))
    plt.title("{} qubit RB surivival probability variance".format(n))
    plt.plot(lengths_CRB[n]-1, varsp_CRB[n],'.-',label='CRB')
    plt.plot(lengths_PRB[n], varsp_PRB[n],'.-',label='PRB')
    plt.legend()
    
    filename =  cwd + "/fig_outs/"+title+" - {}  qubit RB surivival probability variance.pdf".format(n)
    plt.savefig(filename)
    plt.show()

for n in nlist:
    plt.figure(figsize=(15,7))
    plt.title("{} qubit RB average 2-qubit gate count".format(n))
    plt.plot(lengths_CRB[n]-1, a2QC_CRB[n],'.-',label='CRB')
    plt.plot(lengths_PRB[n], a2QC_PRB[n],'.-',label='PRB')
    plt.legend()
    
    filename =  cwd + "/fig_outs/"+title+" - {} qubit RB average 2-qubit gate count.pdf".format(n)
    plt.savefig(filename)
    plt.show()